# Fine‑Tuning Generic Pipeline (LoRA / QLoRA) — Colab

This notebook:
1) Installs dependencies  
2) Fetches your project repo to `/content/project`  
3) Reads YAML configs  
4) Loads model (Kaggle Models or Hugging Face)  
5) Validates dataset  
6) Fine‑tunes with **LoRA/QLoRA**  
7) Exports adapters + metadata  

> **Tip:** If you see organization Drive policy errors, use a personal Gmail or run this on **Kaggle Notebooks**.


In [1]:
%%bash
pip install -q transformers datasets peft accelerate bitsandbytes pyyaml kagglehub jsonschema


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.7/60.7 MB 18.8 MB/s eta 0:00:00


In [3]:

# =========================
# PROJECT FETCHER (Git mode only; no OAuth, no Drive)
# =========================
GIT_URL    = "https://github.com/Srienath2205/genericfinetuningpipeline.git"  # <-- set this
GIT_BRANCH = "main"      # change if you use another branch
GIT_SUBDIR = ""          # optional: if your project is NOT at repo root, set e.g. "generic-pipeline/"

import os, shutil

TARGET = "/content/project"

# Clean target then clone
os.system(f"rm -rf {TARGET}")
if GIT_BRANCH:
    os.system(f"git clone --depth 1 --branch {GIT_BRANCH} {GIT_URL} {TARGET}")
else:
    os.system(f"git clone --depth 1 {GIT_URL} {TARGET}")

# If repo content is inside a subfolder, move it up into /content/project
if GIT_SUBDIR:
    src = os.path.join(TARGET, GIT_SUBDIR)
    assert os.path.isdir(src), f"Subdir '{GIT_SUBDIR}' not found in cloned repo"
    for name in os.listdir(src):
        shutil.move(os.path.join(src, name), os.path.join(TARGET, name))
    shutil.rmtree(os.path.join(TARGET, GIT_SUBDIR), ignore_errors=True)

# basic structure checks
must_dirs = ["configs", "data", "scripts"]
missing = [d for d in must_dirs if not os.path.isdir(os.path.join(TARGET, d))]
if missing:
    raise RuntimeError(f"Missing required folders in {TARGET}: {missing}")

print("[OK] Project ready at:", TARGET)
print("Contents:", os.listdir(TARGET))


[OK] Project ready at: /content/project
Contents: ['configs', 'data', 'requirements.txt', 'notebooks', '.git', 'docs', 'README.md', 'scripts']


In [5]:

import yaml
from datasets import load_dataset
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling
from peft import get_peft_model, LoraConfig, TaskType
import os, sys

# Fallback if BASE_PATH not defined
if "BASE_PATH" not in globals():
    BASE_PATH = "/content/project" if os.path.exists("/content/project") else "/content"

# Ensure the project root is on sys.path so `import scripts.*` works
if BASE_PATH not in sys.path:
    sys.path.insert(0, BASE_PATH)

# (Optional) sanity check
assert os.path.isdir(os.path.join(BASE_PATH, "scripts")), f"'scripts' folder not found under {BASE_PATH}"
print("Using BASE_PATH:", BASE_PATH)


from scripts.prepare_dataset import validate_or_raise
from scripts.load_model_generic import load_model_and_tokenizer
from scripts.export_adapters import export_adapters


def load_yaml(path):
    if not os.path.exists(path):
        raise FileNotFoundError(f"Config not found: {path}")
    with open(path, "r") as f:
        return yaml.safe_load(f)


Using BASE_PATH: /content/project


In [ ]:

model_cfg = load_yaml(f"{BASE_PATH}/configs/model_config.yaml")
train_cfg = load_yaml(f"{BASE_PATH}/configs/training_config.yaml")
data_cfg = load_yaml(f"{BASE_PATH}/configs/dataset_config.yaml")
usecase_cfg = load_yaml(f"{BASE_PATH}/configs/usecase_config.yaml")

print("Configs loaded ✅")


In [7]:
# --- Self-contained dataset load + format cell ---

import os, sys, json
import yaml
from datasets import load_dataset

# --- Ensure BASE_PATH and sys.path for `scripts` imports ---
if "BASE_PATH" not in globals():
    BASE_PATH = "/content/project" if os.path.exists("/content/project") else "/content"
if BASE_PATH not in sys.path:
    sys.path.insert(0, BASE_PATH)

# --- Safe imports of your project utilities ---
from scripts.prepare_dataset import validate_or_raise  # requires sys.path to include BASE_PATH

# --- Helper to load YAML (re-)if needed ---
def load_yaml(path):
    if not os.path.exists(path):
        raise FileNotFoundError(f"Config not found: {path}")
    with open(path, "r") as f:
        return yaml.safe_load(f)

# --- (Re)load configs if they are not in memory ---
if "data_cfg" not in globals():
    data_cfg = load_yaml(f"{BASE_PATH}/configs/dataset_config.yaml")
if "usecase_cfg" not in globals():
    try:
        usecase_cfg = load_yaml(f"{BASE_PATH}/configs/usecase_config.yaml")
    except FileNotFoundError:
        usecase_cfg = {}

print("Using BASE_PATH:", BASE_PATH)

# --- Path resolver works for both relative and absolute paths ---
def resolve_path(p: str) -> str:
    """Return absolute path under BASE_PATH unless already absolute."""
    return p if os.path.isabs(p) else f"{BASE_PATH}/{p}"

# Resolve dataset paths
train_path = resolve_path(data_cfg["train_path"])
eval_path  = resolve_path(data_cfg["eval_path"])

# Validate files (basic JSONL schema checks)
validate_or_raise(train_path)
validate_or_raise(eval_path)

# Load raw dataset
dataset = load_dataset(
    "json",
    data_files={
        "train": train_path,
        "eval":  eval_path,
    }
)

# Format messages into a single text per example
def format_chat(example):
    text = ""
    for msg in example["messages"]:
        if msg["role"] == "user":
            text += f"### Instruction:\n{msg['content']}\n"
        elif msg["role"] == "assistant":
            text += f"### Response:\n{msg['content']}\n"
    return {"text": text}

dataset = dataset.map(format_chat)
print("Dataset prepared. Columns:", dataset["train"].column_names)


Using BASE_PATH: /content/project
[OK] /content/project/data/train.jsonl validated with 3 records
[OK] /content/project/data/eval.jsonl validated with 1 records


Generating train split: 0 examples [00:00, ? examples/s]

Generating eval split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/3 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Dataset prepared. Columns: ['messages', 'text']


In [ ]:
# --- Bootstrap configs & path (safe to re-run any time) ---

import os, sys, yaml

# 1) Ensure project root on sys.path (so `from scripts...` works)
if "BASE_PATH" not in globals():
    BASE_PATH = "/content/project" if os.path.exists("/content/project") else "/content"
if BASE_PATH not in sys.path:
    sys.path.insert(0, BASE_PATH)

assert os.path.isdir(os.path.join(BASE_PATH, "scripts")), f"'scripts' folder not found under {BASE_PATH}"

# 2) Helper to load YAML
def _load_yaml(path: str):
    if not os.path.exists(path):
        raise FileNotFoundError(f"Config not found: {path}")
    with open(path, "r") as f:
        return yaml.safe_load(f)

# 3) Load configs if missing (idempotent)
if "model_cfg" not in globals():
    model_cfg = _load_yaml(f"{BASE_PATH}/configs/model_config.yaml")
if "train_cfg" not in globals():
    train_cfg = _load_yaml(f"{BASE_PATH}/configs/training_config.yaml")
if "data_cfg" not in globals():
    data_cfg = _load_yaml(f"{BASE_PATH}/configs/dataset_config.yaml")
if "usecase_cfg" not in globals():
    usecase_cfg = _load_yaml(f"{BASE_PATH}/configs/usecase_config.yaml")

print("Configs loaded ✅")
print("Using BASE_PATH:", BASE_PATH)

# 4) Now it's safe to import your project helpers
from scripts.prepare_dataset import validate_or_raise
from scripts.load_model_generic import load_model_and_tokenizer
from scripts.export_adapters import export_adapters
``

In [8]:

# Load model/tokenizer

tokenizer, model, model_path = load_model_and_tokenizer(
    model_source=model_cfg["model_source"],
    model_name=model_cfg["model_name"],
    quantization=model_cfg.get("quantization", "4bit"),
    device_map=model_cfg.get("device_map", "auto"),
    torch_dtype=model_cfg.get("torch_dtype", "bfloat16"),
    for_training=(train_cfg["method"] in ["lora", "qlora"]),
)

tokenizer.pad_token = tokenizer.eos_token
if hasattr(model, "config"):
    model.config.pad_token_id = tokenizer.eos_token_id

# Optional: gradient checkpointing to reduce memory
if hasattr(model, "gradient_checkpointing_enable"):
    try:
        model.gradient_checkpointing_enable()
    except Exception:
        pass

print("Model loaded:", model_path)


NameError: name 'model_cfg' is not defined

In [ ]:

MAX_LEN = train_cfg.get("max_seq_length", 512)

def tokenize(example):
    return tokenizer(
        example["text"],
        truncation=True,
        padding="max_length",
        max_length=MAX_LEN,
    )

tokenized = dataset.map(
    tokenize,
    remove_columns=dataset["train"].column_names,
)
print("Tokenization complete. Example keys:", tokenized["train"].column_names)


In [ ]:

if train_cfg["method"] in ["lora", "qlora"]:
    lora_cfg = LoraConfig(
        r=train_cfg["lora"]["r"],
        lora_alpha=train_cfg["lora"]["alpha"],
        target_modules=train_cfg["lora"]["target_modules"],
        lora_dropout=train_cfg["lora"]["dropout"],
        task_type=TaskType.CAUSAL_LM,
    )
    model = get_peft_model(model, lora_cfg)
    print("LoRA/QLoRA adapters attached.")
else:
    print("Full finetuning mode.")


In [ ]:

from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="/content/output",
    per_device_train_batch_size=train_cfg["batch_size"],
    gradient_accumulation_steps=train_cfg["gradient_accumulation_steps"],
    num_train_epochs=train_cfg["epochs"],
    learning_rate=float(train_cfg["learning_rate"]),
    logging_steps=train_cfg["logging_steps"],
    eval_steps=train_cfg["eval_steps"],
    save_steps=train_cfg["save_steps"],
    save_total_limit=2,
    evaluation_strategy="steps",
    fp16=True,
    report_to="none",
    group_by_length=True,
)

collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["eval"],
    tokenizer=tokenizer,
    data_collator=collator,
)

trainer.train()
print("Training complete.")


In [ ]:

export_adapters(
    model,
    usecase_name=usecase_cfg["usecase_name"],
    extra_meta={"model": model_cfg["model_name"]},
)
